In [1]:
%pip install -q -U scikit-learn pandas numpy joblib scipy optuna matplotlib ipywidgets xgboost lightgbm pywavelets seaborn


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install necessary packages (if not already installed)
# Uncomment and run the following lines if necessary:
# %pip install -q -U scikit-learn pandas numpy joblib scipy optuna matplotlib ipywidgets xgboost lightgbm pywavelets seaborn

# Imports
import json
import pandas as pd
import numpy as np
import joblib
from scipy.fft import fft
from scipy.signal import butter, filtfilt
import pywt  # For wavelet transforms
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import IntSlider, ToggleButtons, Dropdown, interactive_output, HBox, VBox, Output
import time


In [3]:

# Step 1: Load and Correct the Data

# Load the data
with open('cycling_data2.json', 'r') as f:
    data = json.load(f)

# Define gear ratios and wheel circumference
gear_ratios = {
    1: 38.0 / 11.0,
    2: 38.0 / 15.0,
    3: 38.0 / 18.0,
    4: 38.0 / 21.0,
    5: 38.0 / 24.0,
    6: 38.0 / 34.0,
    # Add all your gear ratios
}
wheel_circumference = 2.19999  # in meters

# Flatten the data and recalculate cadence
records = []
for session in data:
    for point in session['data']:
        gear = point['gear']
        speed = point['speed']  # in m/s
        gear_ratio = gear_ratios.get(gear, None)
        if gear_ratio and speed > 0:
            cadence = (speed / wheel_circumference) * gear_ratio * 60
        else:
            cadence = 0.0
        point['cadence'] = cadence
        records.append({
            'timestamp': point['timestamp'],
            'accel_x': point['accelerometerData']['x'],
            'accel_y': point['accelerometerData']['y'],
            'accel_z': point['accelerometerData']['z'],
            'speed': speed,
            'cadence': cadence,
            'gear': gear,
            'terrain': point['terrain'],
            'is_standing': point['isStanding']
        })

# Convert to DataFrame and Ensure Temporal Order

df = pd.DataFrame(records)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.sort_values('timestamp', inplace=True)
df.reset_index(drop=True, inplace=True)


In [4]:

# Step 2: Define Functions for Feature Extraction and Signal Processing

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs  # Nyquist Frequency
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def extract_features(window, include_fft=True, apply_filter=False, include_wavelet=False):
    # Extract accelerometer data
    accel_x = window['accel_x'].values
    accel_y = window['accel_y'].values
    accel_z = window['accel_z'].values

    if apply_filter:
        # Apply band-pass filter
        fs = 50  # Sampling frequency
        lowcut = 0.5
        highcut = 3.0  # Adjust based on expected cadence frequency range
        accel_x = bandpass_filter(accel_x, lowcut, highcut, fs)
        accel_y = bandpass_filter(accel_y, lowcut, highcut, fs)
        accel_z = bandpass_filter(accel_z, lowcut, highcut, fs)

    features = []

    # Time-domain features
    for accel in [accel_x, accel_y, accel_z]:
        features.extend([
            np.mean(accel),
            np.std(accel),
            np.var(accel),
            np.min(accel),
            np.max(accel),
            np.median(accel),
            np.percentile(accel, 25),
            np.percentile(accel, 75),
        ])

    # Frequency-domain features
    if include_fft:
        for accel in [accel_x, accel_y, accel_z]:
            fft_values = fft(accel)
            fft_magnitude = np.abs(fft_values)[:len(fft_values)//2]
            features.extend([
                np.mean(fft_magnitude),
                np.std(fft_magnitude),
                np.max(fft_magnitude),
                np.argmax(fft_magnitude),  # Dominant frequency component
            ])

    # Wavelet features
    if include_wavelet:
        for accel in [accel_x, accel_y, accel_z]:
            coeffs = pywt.wavedec(accel, 'db1', level=3)
            # Use approximation coefficients at the third level
            cA3 = coeffs[0]
            features.extend([
                np.mean(cA3),
                np.std(cA3),
            ])

    # Add speed feature
    features.append(window['speed'].mean())

    return features


In [14]:

# Step 3: Define the Interactive Visualization Function with Dynamic Model Training

# Function to calculate gear from cadence and speed
def calculate_gear(cadence, speed, wheel_circumference, gear_ratios):
    if speed == 0:
        return np.nan
    # Calculate gear ratio
    gear_ratio_calculated = (cadence * wheel_circumference) / (speed * 60)
    # Match to the closest gear ratio
    gear_ratio_values = np.array(list(gear_ratios.values()))
    gear_numbers = np.array(list(gear_ratios.keys()))
    idx = (np.abs(gear_ratio_values - gear_ratio_calculated)).argmin()
    predicted_gear = gear_numbers[idx]
    return predicted_gear

# Cache models to avoid retraining if parameters are the same
model_cache = {}

def interactive_plot(window_size, window_step, include_fft, include_wavelet, scaler_name, apply_filter, use_pca, model_type, window_index):
    params_key = (window_size, window_step, include_fft, include_wavelet, scaler_name, apply_filter, use_pca, model_type)
    start_time = time.time()
    if params_key in model_cache:
        data = model_cache[params_key]
    else:
        # Prepare data
        X = []
        y_cadence = []
        y_gear = []
        X_raw = []

        # Generate windows
        for start_idx in range(0, len(df) - window_size, window_step):
            end_idx = start_idx + window_size
            window = df.iloc[start_idx:end_idx]

            # Extract features
            features = extract_features(window, include_fft=include_fft, apply_filter=apply_filter, include_wavelet=include_wavelet)
            X.append(features)
            y_cadence.append(window['cadence'].mean())
            y_gear.append(window['gear'].mode()[0])

            # Store raw data for visualization
            X_raw.append({
                'accel_x': window['accel_x'].values,
                'accel_y': window['accel_y'].values,
                'accel_z': window['accel_z'].values,
                'speed': window['speed'].mean(),
            })

        X = np.array(X)
        y_cadence = np.array(y_cadence)
        y_gear = np.array(y_gear)

        # Handle missing values
        imputer = SimpleImputer(strategy='mean')
        X = imputer.fit_transform(X)

        # Select scaler
        if scaler_name == 'StandardScaler':
            scaler = StandardScaler()
        elif scaler_name == 'MinMaxScaler':
            scaler = MinMaxScaler()
        else:
            scaler = RobustScaler()

        # Scale features
        X_scaled = scaler.fit_transform(X)

        # Apply PCA if selected
        if use_pca:
            pca = PCA(n_components=0.95)
            X_scaled = pca.fit_transform(X_scaled)

        # Time-based splitting
        split_index = int(0.8 * len(X_scaled))
        X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
        y_train_cadence, y_test_cadence = y_cadence[:split_index], y_cadence[split_index:]
        y_test_gear = y_gear[split_index:]
        X_test_raw = X_raw[split_index:]

        # Select and train model
        if model_type == 'RandomForest':
            model = RandomForestRegressor(n_estimators=100, random_state=42)
        elif model_type == 'LightGBM':
            model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=42)

        # Training the model
        with output:
            output.clear_output(wait=True)
            print("Training model, please wait...")
        model.fit(X_train, y_train_cadence)

        # Store the model and data in cache
        data = {
            'model': model,
            'scaler': scaler,
            'X_test': X_test,
            'y_test_cadence': y_test_cadence,
            'y_test_gear': y_test_gear,
            'X_test_raw': X_test_raw,
        }
        model_cache[params_key] = data

    # Retrieve data
    model = data['model']
    X_test = data['X_test']
    y_test_cadence = data['y_test_cadence']
    y_test_gear = data['y_test_gear']
    X_test_raw = data['X_test_raw']

    idx = window_index  # Each index corresponds to one window

    # Get the test sample
    X_sample = X_test[idx].reshape(1, -1)
    y_actual_cadence = y_test_cadence[idx]
    y_actual_gear = y_test_gear[idx]
    X_raw_sample = X_test_raw[idx]

    # Predict using the model
    y_pred_cadence = model.predict(X_sample)[0]

    # Calculate predicted gear
    speed = X_raw_sample['speed']
    predicted_gear = calculate_gear(y_pred_cadence, speed, wheel_circumference, gear_ratios)

    # Plot accelerometer data
    fig, axs = plt.subplots(4, 1, figsize=(12, 16))

    accel_x = X_raw_sample['accel_x']
    accel_y = X_raw_sample['accel_y']
    accel_z = X_raw_sample['accel_z']
    time_accel = np.arange(len(accel_x)) / 50  # Assuming 50 Hz sampling rate

    axs[0].plot(time_accel, accel_x, label='Accel X')
    axs[0].plot(time_accel, accel_y, label='Accel Y')
    axs[0].plot(time_accel, accel_z, label='Accel Z')
    axs[0].set_title('Accelerometer Data')
    axs[0].set_xlabel('Time (s)')
    axs[0].set_ylabel('Acceleration')
    
    axs[0].legend()

    # Plot actual vs predicted cadence
    axs[1].bar(['Actual Cadence', 'Predicted Cadence'], [y_actual_cadence, y_pred_cadence], color=['blue', 'orange'])
    axs[1].set_title('Actual vs. Predicted Cadence')
    axs[1].set_ylabel('Cadence (RPM)')
# set the y axis to be the max of the actual and predicted cadence
    axs[1].set_ylim([0, max(y_actual_cadence, y_pred_cadence) + 10])

    # Plot actual gear vs predicted gear
    axs[2].bar(['Actual Gear', 'Predicted Gear'], [y_actual_gear, predicted_gear], color=['green', 'red'])
    axs[2].set_title('Actual vs. Predicted Gear')
    axs[2].set_ylabel('Gear Number')
    axs[2].set_ylim([0, 7])

    # Plot speed
    axs[3].bar(['Speed'], [speed], color='purple')
    axs[3].set_title('Speed')
    axs[3].set_ylabel('Speed (m/s)')
    axs[3].set_ylim([0.0, speed + 1.0])

    plt.tight_layout()
    plt.show()

    with output:
        output.clear_output(wait=True)
        print(f"Model trained in {time.time() - start_time:.2f} seconds.")


In [16]:

# Step 4: Create Interactive Widgets

# Widgets for parameters
window_sizes = [10,30,50,100, 200, 300,500]
window_steps = [5,20,50, 100]
include_fft_options = [('Yes', True), ('No', False)]
include_wavelet_options = [('Yes', True), ('No', False)]
scaler_options = ['StandardScaler', 'MinMaxScaler']
filter_options = [('Yes', True), ('No', False)]
pca_options = [('Yes', True), ('No', False)]
model_types = ['RandomForest', 'LightGBM']

window_size_slider = Dropdown(
    options=window_sizes,
    value=200,
    description='Window Size:',
)

window_step_slider = Dropdown(
    options=window_steps,
    value=50,
    description='Overlap:',
)

include_fft_toggle = ToggleButtons(
    options=include_fft_options,
    value=True,
    description='Include FFT:',
)

include_wavelet_toggle = ToggleButtons(
    options=include_wavelet_options,
    value=False,
    description='Include Wavelet:',
)

scaler_dropdown = Dropdown(
    options=scaler_options,
    value='StandardScaler',
    description='Scaler:',
)

filter_toggle = ToggleButtons(
    options=filter_options,
    value=False,
    description='Apply Filter:',
)

pca_toggle = ToggleButtons(
    options=pca_options,
    value=False,
    description='Use PCA:',
)

model_type_dropdown = Dropdown(
    options=model_types,
    value='RandomForest',
    description='Model Type:',
)

# Function to update the window index slider based on selected parameters
def update_window_index(*args):
    # Since the data length depends on window_size and window_step, we need to calculate it
    window_size = window_size_slider.value
    window_step = window_step_slider.value
    num_windows = (len(df) - window_size) // window_step
    window_index_slider.max = max(num_windows - 1, 0)

# Window index slider
window_index_slider = IntSlider(min=0, max=10, step=1, description='Window Index')

# Observe changes to update window index slider max value
window_size_slider.observe(update_window_index, 'value')
window_step_slider.observe(update_window_index, 'value')

# Initialize window index slider max value
update_window_index()

# Output widget to display training messages
output = Output()

# Interactive output
ui = VBox([
    HBox([window_size_slider, window_step_slider]),
    HBox([include_fft_toggle, include_wavelet_toggle]),
    HBox([scaler_dropdown, filter_toggle]),
    HBox([pca_toggle, model_type_dropdown]),
    window_index_slider,
    output
])

out = interactive_output(
    interactive_plot,
    {
        'window_size': window_size_slider,
        'window_step': window_step_slider,
        'include_fft': include_fft_toggle,
        'include_wavelet': include_wavelet_toggle,
        'scaler_name': scaler_dropdown,
        'apply_filter': filter_toggle,
        'use_pca': pca_toggle,
        'model_type': model_type_dropdown,
        'window_index': window_index_slider
    }
)

display(ui, out)

Output()